In [4]:
import pandas as pd 
import re

import numpy as np

pd.set_option('display.max_rows', None)

####한국어 형태소 분석기####
from konlpy.tag import Kkma, Komoran, Okt, Hannanum #Mecab은 윈도우에서 작동 불가능
okt = Okt()

### 기사 감성분석한거 dataframe으로 만들기

df = pd.read_csv('positive_negative_text_result.txt', sep='\t', header=None)

df_price=pd.read_excel('../../data/가격데이터_증감비교_라벨링_가격등락비율.xlsx')

df_price = df_price.drop(df_price.columns[[0]], axis=1)

df.columns=['article', 'sentiment']
df['Week_number']=0

index_list = df[df['article'].str.contains('번쨰')].index

df['article'] = df['article'].replace('NULL','', regex=True)

df[df['article'].str.contains('번쨰')]

,article,sentiment,Week_number
0,0번쨰 기사입니다,neutral,0
42,1번쨰 기사입니다,neutral,0
113,2번쨰 기사입니다,neutral,0
179,3번쨰 기사입니다,neutral,0
235,4번쨰 기사입니다,neutral,0
354,5번쨰 기사입니다,neutral,0
397,6번쨰 기사입니다,neutral,0
398,7번쨰 기사입니다,neutral,0
430,8번쨰 기사입니다,neutral,0
508,9번쨰 기사입니다,neutral,0


In [5]:
# 인덱스를 이용하여 208주 그룹형태로 나누기 
for i in range(len(index_list)):
    index=index_list[i]
    if i!=207:
        next_index=index_list[i+1]
    else:
        next_index=-1
    df['Week_number'].iloc[index:next_index]=i+1

C:\Users\jaemin\AppData\Local\Temp\ipykernel_2640\2446332499.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Week_number'].iloc[index:next_index]=i+1


In [6]:
missing_numbers = set(range(1, 208)) - set(df['Week_number'])
print(missing_numbers)

set()


In [7]:
df['sentiment']=df['sentiment'].replace('positive',1)
df['sentiment']=df['sentiment'].replace('negative',-1)
df['sentiment']=df['sentiment'].replace('neutral',0)

In [15]:
df

,article,sentiment,Week_number,article_split
0,0번쨰 기사입니다,0,1,번쨰 기사 이다
1,돈 호조로 입식 는 영향새해 역시 증가세 이어,1,1,돈 호조 입 식 늘다 영향 새해 역시 증가 세 이다
2,연간 출하물량인 1천654만마리를 넘어섰다,0,1,연간 출하물량 마리를 넘어서다
3,연말까지 1천670만두에 이를 것으로 예상된다,0,1,연말 두에 이르다 것 예상 되다
4,연속 사상최고의 출하두수 기록을 세우게 됐다,1,1,연속 사상 최고 출하 두수 기록 세우다 돼다
5,결과로 새해도 출하 증가세가 이어질 전망이다,1,1,결과 새해 출하 증가 세 이어지다 전망
6,돈 하락 시 경영 악화 우려이유 후 육성에 관심 집중을양돈농가 중 절반이 넘는 농가...,-1,1,돈 하락 시 경영 악화 우려 이유 후 육성 관심 집중 양돈 농가 중 절반 넘다 농가...
7,농가의 생산 성적을 최종 취합한 결과,0,1,농가 생산 성적 최종 취합 결과
8,가격 하락시 이들 농가들의 경영악화가 우려되고,-1,1,가격 하락 시 이 들 농가 들 경 영 악화 우려 되다
9,농가수 비율 분석 자료에 따르면 3천320농가 중,0,1,농 가수 비율 분석 자료 따르다 농가 중


In [8]:
stem_list2=[]
stem_list3=[]
for text in df['article']:
    malist=okt.pos(text, norm=True, stem=True)
    r=[]
    for word in malist:
        if not word[1] in ["Josa","Eomi","Punctuation", "Number",'\r','\t','unknown']:
            r.append(word[0])
    rl=(" ".join(r)).strip()
    stem_list2.append(rl)
    print(rl)

번쨰 기사 이다
돈 호조 입 식 늘다 영향 새해 역시 증가 세 이다
연간 출하물량 마리를 넘어서다
연말 두에 이르다 것 예상 되다
연속 사상 최고 출하 두수 기록 세우다 돼다
결과 새해 출하 증가 세 이어지다 전망
돈 하락 시 경영 악화 우려 이유 후 육성 관심 집중 양돈 농가 중 절반 넘다 농가 MSY 두 미만 기록
농가 생산 성적 최종 취합 결과
가격 하락 시 이 들 농가 들 경 영 악화 우려 되다
농 가수 비율 분석 자료 따르다 농가 중
비교 하다 보다 때 MSY 두 미만 비율
소 폭 늘어나다
이유 후 육 성률 낮아지다 영향 풀이 되다
이후 약세 역 부족 지난해 돼지 값 전 년 대비 오르다 연속
가량 오르다
이상 기록 하다 호황 이다 가다
도인 강세 뒷받침 결과
문턱 넘다 못 하다
이후 물량 증가 강세 이다
최고 종개 협 신규 종돈 장 입 식 일반 농장 직수입 여파 작년 종돈 수입 물량 크게 증가 하다 두를 넘어서다
지난해 종돈 수입 두수 통관 기준 두 전 년 두 비다
두 이다 사상 번째 많다 물량
두 가장 많다 그 다음 프랑스 두
품종 별로 보다
수 두 △ 버크셔
업체 곳 두르다 수입 하다
특히 일반 농장 수입 물량 전 년
수입 물량 하다 몫 하다
값 하락 세 올해 이어지다 전망
지난해 미치다 못 하다 것 내다보다
전 수준 밑돌 것 예상 하다
전 년 대비 상승세 기록 하다 부터 하락 세로 전환 돼다 농 경연 전망 따르다 적어도 내년 상반기 이 같다 흐름 내내 지속 되다 것
늘 과 을 제외
이 따르다 상반기 돼지 출하물량
많다 것 추산 돼다
한 돈 생산 늘다 수입량 감소
또 이 기간 돼지 사 육 두수 전 년 대비
해롭다 만들다
이중 축산 사육 환경 근본 적 개선 하다 것 강조 하다
산란 계 신규 농장 보다 넓어지다 사 육 밀도 기준 적용 하다 것 강조 하다
사육 환경 표시 제도 도입 하다
관계 부처 협의 거치다 구 제 방안 마련 하다
번쨰 기사 이다
만두 넘다 종개 협 작년 두 전 년 증가 최근 연속 후보 돈
실적 분석 결과 F 후보 돈
매 두수 두 전 

최근 농촌 경 제 연구원 국제 곡물 호를 통해 기 사료 용 곡물 수입 단가 지수 로 전 분기
더 오르다 것 전망 하다
국제 곡물 가격 오르다 시작 오다 이후 구매 물량 이기 때문
오르다 것 점 치다
비 하다 낮다 수준 이기다 하나 최근 국제 곡물 시장 흐름 보다 긴장 늦추다 순 없다
콩 가격 상승 미국 밀 작황 부진 등 따르다 강세 형성
오르다 기는 로 기보다
가격 상승 가능성 있다 지적 하다
가격 하락 전망 농가 들 모돈 사육 의향 높다 것 나타나다
상승 것 조사 돼다
가격 올해 하락 세 이나 생산비 이상 유지 하다 것 전망 돼다 때문 풀이 되다
이르다 뒷받침 하 듯 올해 후보 돈 판매 두수 크게 늘다
이동 등록 현황 보다 올해 이동 판매 두수 총 두 작
증가 하다
총 사 육 두수 백만 근절 예상 되다
화농 탓
목살 가격 하락 일로 처 하다
부위 들 중 가장 하락 폭 크다
연속 하락 하다
가까이 가격 떨어지다
주목 받다 되다 것 하다 원인 파악 되다
있다 목심 부위 화농 하다 원인
지속 적 해당 문제 이슈 화 되다 있다 목심 가격 회복 더욱 어렵다 하다
대책 마련 필요하다
양돈 농가 들 반감 커지다
농림축산식품부 지난해
패러다임 전환 하다 가정 일부 시설 용어 역시 전환 하다
용어 변경 농축 산부 촉구 하다
순화 되다 용어 변경 하다 사용 하다 것 요구
약세 지 속 향후 돼지 값 출하 두 수가 증가 하다 전 년 대비 약세 지속 되다 것 전망 돼다
하락 하다 것 내다보다
지난해 수준 회복하다 못 하다 것
지난해 낮다 수준 형성 하다 것 예상 하다
따르다 결과 설명 하다
증가 마
기록 하다 것 추산 하다
모돈 입 식 증가 돼지 사 육 두 수도 지속 적 지난해 많다 수준 유지 되다 것 전망 돼다
많다 것 내다보다
가량 감소 톤을 기록 하다 것 전망 돼다
PED 강타
돼지 질병 창궐 경고 되다
여름 출하 예상 늘 않다 수 있다 전망 나오다 정도
이 같다 우려 나오다
℃ 로 평년 대비
특히 온도 낮다 기승 부리다 PED
더 심각하다 수준 진단
문제 PED 그 자체 많

번쨰 기사 이다
모돈 작년 늘다
돼지 사 육 두수 지난해 증가 것 나타나다
통계청 지난 기 가축 사육
늘다
이 가운데 모돈 수 마리로 과 같다 지난해 에 비다
가량 두수 많다
이처럼 전체 두수 늘다
각각 감소 하다
감소 하다
늘다
이로써 기준 농장 당 사 육 두수 마리 지난해 동월
↑ △ 마리 이상 마리 전 년비
소 폭 오르다
한편 이번 돼지 사 육 두수 통계 표본 농가 범위 기존 농림
농가 함께 법인 포함 되다 기존 통계 차이 발생 하다
올해 을 넘기다 힘들다 전망
돼지고기 생산 량 증가 돼지 지 육 가격 전 년 약세 보이다 것 전망 하다
하락 하다 것 예상 하다
이다 탕박 기준 제주 가격 포함 수치 제주 가격 제 외한 다 면 이보 더 하락 가격 형성 되다 것 보이다
출하 두수 마리가 예측 되다
수준 연중 최고 형성 하다
전 년 낮다 형성 되다 것 예측 돼다
들어가다 가운데 수입량 예상 빗나가다
가량 적다
늘다
이렇게 수입 급증 것 이다
전망 이유 국내 돼지 출하 증가 돼지 값 하락 때문 이다
하락 하다
수입 이처럼 증가 이유 수입 부위 찾다 수 있다
값 하락 하다 전후 부위 강세 보이 수 입산 대한 수요 증가 것
상승 하다
급증 하다 뒷다리 지난해 여 톤 올해 톤을 넘기다 배 가량 증가 하다
여기 세계 주요 국가 들 돼지 값 하락 한몫 하다
떨어지다 약세 형성
환율 받치다 주다
올해 원화 강세 보이다
이 같다 국제 돼지 값 하락 원화 강세 수입 가격 반영 돼다
가량 하락 하다
향후 돼지고기 수입 전망 낙 관할 수 없다
국내 수입 더 부추기다 수 있다 서다
세계 적 올해 돼지고기 생산량 증가 하다 돼지 값 하락 하다 것 전망 향후 밉다
번쨰 기사 이다
번쨰 기사 이다
■ 한 돈
하락 돼지 값 오다 들다
지난해 동월 대비 하락 폭도 가장 크다
오르다
계절 적 돼지고기 소비 살아나다 시기 때문
낮다
이후 돼지 값 전 년 대비 낮다 수준 지속 되다 가운데 하락 폭 가장 크다
비교 하다 출하 거의 줄 않다
돼지 값 하락 폭 키우다 것 분석 되다
돼지 출하 두 수가 

시작 하다 지금 돼지 값 흐름 감안 하다 올해 다소 예 외적인 상황 셈
돼지 값 보다 낮다 경우 지난 이후 만이다
공교 롭 도 은 돼지고기 수입 자유 화 되다 해 하다
돼지 값 이처럼 하락 것 무엇 출하 많다 것 하다 원인
마리에 비 하다 다소 줄다
시장 부담 되다 것 보이다
소비 저조하다
올해 이렇다 하다 김장 철 특수 감지 하다 어렵다
한 돈 시장 적지 않다 부담 되다 것 전망 되다
저작권 자
양돈 타임스 무단 전 재 및 재 배포 금지 임정은 다른 기사 보기 번쨰 기사 이다
돼지고기 수입량 톤이상
기록 하다 올해 수입량 톤을 넘어서다
각각 늘다
톤 이다 네 번 째 많다
요인 따르다 것 보이다
수입량 톤을 넘기다 사상 최고 치 기록 경신 하다
이상 되다 가능성 높다
수입량 부위 별로 보다 역시 연말 수요 많다 삼겹살 에 비다 무려
그렇다 말로 보다 앞다리 톤으로 일 년 전보
이상 늘다
최근 국내 등심 수요 증가 하다 한 돈 등심 고가 형성
칠레
톤 순 기록 하다
저작권 자
양돈 타임스 무단 전 재 및 재 배포 금지 임정은 다른 기사 보기 번쨰 기사 이다
수 매 비축 카드 꺼내다 들다
농가 피해 심화 되다 따르다 최근 긴급 회의 개최
비용 지원 하다 것
비용 지원 다
자 조금 관리 위원회 수급 위 결정 사항 토대 도매시장 후지 수 매 비축 사업 위 계획 수립
농림축산식품부 최종 승인 남아 있다
번쨰 기사 이다
번쨰 기사 이다
중단 하다 것 촉구 하다
협회 최근 극심하다 경 기 불황 급격하다 도인 폭락 농가 경영 위기 감 고조 되다
이 협회
성
확장 진행 되다 경우 이다 양돈 농가 들 생존권 심각하다 위협 하다 하다 행위
협회
농가 바램 짓 밟다 행위
이 협회 기업 자본 무분별 농장 매입 시도 계속 되다 경우 해당 업체 실명 농장 매입 내용 공개 하다 강조 하다
번쨰 기사 이다
년 돼지고기 수입량 톤을 처음 넘기다 사상 최고 치르다 기록 하다
늘다
이로써 수입량 톤보다
한 돈 생산 량 증가 수입량 증가 이유 수요 공급 측면 모두 찾다 수 있다
중국 수입 줄이다


며 ASF 의 위협 코 앞 다가오다 상황 APVS 의 즉각 취소 촉구 하다
전염 우려 전격 적 취소 되다 사례 교훈 삼 아야 하다 것
바 있다 입장 변화 대한 귀 추가 주목 되다
국내 유입 막다 위해 지정 특별 관리 지역 개 곳 확대 되다
양돈 농가 잔반 자가 급여 금지 키로 하다
대응 강화 방안 확정 하다
동두천 추가 확대 키로 하다
이 함께 남다 음식물 자가 급여 농가 개소
이르다 금지 한다는 계획
범 부처 총력 대응 위해 농림축산식품부 차관 단장
분야 별로 협력 체계 구축 한다는 계획
지자체 합동 영상 회의 매일 개최 하다 방역 상황 점검 이 달 중 접경 지역 농가 관계 기관 합동 가상 방역 훈련 실시 키로 하다
이 함께 민통선 이북 지역 멧돼지 포획 강화하다 포획 되다 멧돼지 ASF 검사 키로 하다
소독
혈청 검사 결과 전 두수 ASF 음성 확인 돼다
농가 울타리 설치 지원 하다 줄 것 요청 하다
정부 건의 하다
기존 사업 신청 절차 간소화
즉시 수렵 허용 되다 관련 절차 간소화 하다 하다
저작권 자
양돈 타임스 무단 전 재 및 재 배포 금지 임정은 다른 기사 보
번쨰 기사 이다
가격 약세 상반기 양돈 장 수익 전 년 대비 감소 가운데 생산 성도 정체 되다 올해 전반 적 양돈 경영 빨갛다 불 켜지다
하락 하다
겨우 지난해 생산비 수준 유지
오다 상반기 농가 수익 본전
각각 적자 형성 것 나타나다
가격 상승 폭 낮다 수익 올 리기 역 부족
전망 되다 올해 양돈 경영 수지 전 년 비다 크게 낮아지다 것 예측 되다
이렇다 상황 생산 성 낮다 농가 경영 위기 직면 하다 것 예측 되다
평균 하위 농가 들 생산 성적 지난해 더 하락 것 분석 돼다 때문
최근 한 돈 소비 형태 볼 때 하
기도 상승 하다 어렵다
번쨰 기사 이다
캠핑 열기 확산 불 지피다
캠핑 페스티벌 성황리 개최 하다
이번 행사 캠핑 문화 즐기다 캠핑 족 들 우리 돼지 한 돈 맛 우수 성 알리
상당 한 돈 패키지 한 돈 기념품 등 푸다 짐 상품 제공 하다
많다 참가자 들 크다 호응 얻다
선물 세트 증

...

벌금 과태료 적용 과도 규제 성 토 하다
해결 하다 수 없다 음 분명하다 하다
번쨰 기사 이다
폐업 원금 대상 선정 돼다
관련 기사 호 면 ‘ http
이의 신청 접수 키로 하다
또 돼지고기 수입 기여 도
수입량 급격하다 증가 하다 가격 하락 피해 입다 품목 생산자 가격 하락 일정 부분 지원 하다 제도
폐업 원금 대상 선정 하다
피해 보전 제도 직 불금 대상 품목 선정 되다 못 하다
FTA 체결 국 수입 량 요건
대부분 충족 하다 국내 가격 요건 충족 하다 못 하다 때문
요건 충족 하다
초과 하다 FTA 체결 국 수입량 톤 기준 톤 직전 협정 상대 국 수입 량 중 최저
이상 기록 하다 기준 부합 하다
폐업 지원 대상 품목 최종 확정 하다 예정
지급 대상자 등 세부 내용 최종 확정 하다 계획
저작권 자
원산지 표시 감시 강화 되다
소비자 들 안심
식품 배달 음식 주문 시 원산지 표시 확인 배송 수령 시 원산지 표시 여부 꼼꼼하다 확인 하다 것 당부 하다
저작권 자
농가 들 정부 과도 방역 대책 크게 반발 하다
거리 나서다
한 돈 협회 및 ASF
희생 농가 비상 대책 위원회 등 여명 한 돈 농가 들
방역 조치 규탄 하다
항 요구 하다
과도 방역 조치 항의 하다
투쟁 하다 것
피해 지역 재입 식 허용
농가 생존권 사수 대 정부 투쟁 선포 기자회견 문 더 이상 차다 수 없다
생업 포기 직전 한계 상태 도달 하다
농가 출입 차량 통제 밀다
이다 농가 현실 무시 탁상 행정 더 이상 차다 수 없다
농가 들 투쟁 하다 것 온 국민 들 선언 하다
희생 농가 대한 조속 재입 식 허용 하
농가 수많다 관련 종사 자 들이다 흘리다 피 눈물 기억 하라
애 태우다
야생 멧돼지 생존권 못 개
산업 보호 하다 위함 자명하다 만큼 정부 하루빨리 재입 식 기준 일정 제시 하
최소 을 기다리다 하다
이 시간 버티다 수 있다 지다 한계 상황
발생 손실 보상 해주다 것 마땅하다 것
둘째
‘ 환경부 직무 유기
광역 울타리 밖 야생 멧돼지 ASF 가다 잇따르다 발견 되다
환경부 야생 멧돼지 제어 하다

환경부
발생 지역 집중 수색 감염 확산 가속 하다 우려 있다 몰이 식 수렵 중단
사 육 돼지 야생 멧돼지 방역 분리 하다 하다 목소리 높이
축산 회관 대 회의실 돼지고기 유통 질서 확립 위 위 상생 협력 공동 협약 MOU
협회 농산물 품질 관리원 돼지고기 유통 질서 확립 위해 손 잡다
이력 관리 상호 발전 관 상생 협력 공동 협약 MOU
홍보 등 협력 하다 하다
상호 협력 키로 하다
소비자 단체 등 집합 교육 강화하다
등 설치 키로 하다
서로 협력 키로 하다
제공 하다 등 상호 협력 키로 하다
번쨰 기사 이다
임시 공휴일 특수 모두 침수 돼다
급락 하다 휴가 철 무색 케 하다 약세 기록 하다
상승세 기록 하다 최근 흐름 비교 하다 한참 못 미치다 상승 폭
장마 휴가 철 소비 피크 이루다 하다 중순 이어진 때문
임시 공휴일 이어지다 광복절 연휴 한 돈 시장 반전 이끌다 못 하다
단계 격상 되다 소비 시장 더욱 움츠러들다 하다
공급 돼지 값 부담 요인 돼다
저작권 자
기사 보기 야생 멧돼지 아프리카 돼지 열병 검 출현 황
기존 발생 지르다 벗어나다 사례 들 잇따르다
최근 환경부 춘천시 사북면 오탄 리 인근 민
바이러스 검 추다 돼다
화천군 사내면 용담 리 로부터 남동쪽 ㎞ 떨어지다 있다
강원도 춘천 ASF 감염 되다 멧돼지 발견 되다 것 이번 처음
어디 더 확산 되다 우려 커지다
장군산 일대 로부터 확산 해오다 것 추정
강화 되다 방역 조치 시행 키로 하다
즉시 점검 취약 구간 발견 시 즉시 보강 키로 하다
출입문 등 확산 우려 있다 구간 포획틀 포획 트랩 집중 설치 전담 관리원 배치 하다 개체 수 줄이다 하다
진입 로 소독 하다 등 관리 강화하다 양돈 장 내 유입 차단 적극 나서다 하다
저작권 자
연속 두 이하 기록 하다 것 전망 되다
가량 줄다
이 따르다 올해 수입 물량
미만 머무르다 전망
가격 약세 하다 요인 분석 되다
수입 비중 역전 돼다
수입 하다 일반 양돈 장 수입 전체 비중 절반 이상 넘어서다 것
종돈장 들 도인 약세 전망 따르다 종돈 수입 유보
올해

현재 부정 적 일 것 응답 가구 수가 많다 설명
가량 높다 것 내다보다
수준 못 미치다 것 분석
경기 침체 시 도 가격 전망
돼지 값 형성 중요하다 공 급량 올해 한 돈 줄
감소 하다 것 추산 돼다
농 경연 예상 하다
돼지 도축 마릿수 전망
저작권 자
양돈 타임스 무단 전 재 및 재 배포 금지 임정은 다른 기사 보기 번쨰 기사 이다
축산 분야 공익 직불제 도입 추진 되다 전망
농어촌 특별 위원회 최근 ‘ 축산 농가 경영 안정화 방안
환경 조화 되다 지속 가능하다 농축 산업 실현 목표 추진 되다
특히 농특위 축산 경영 안정화 위해 축산 분야 공익 직불제 도입 검토
지역 자원 기반 경축 순환 활성화 사업 참여 하다 농가 대상 직 불금 지급 하다 농가 들 사업 참여 유도 하다 것
포함 시키다 축산 분야 공익 직불제 지급 가능 법 개정 추진 되다
농특위 적정 사육 두수 관리제 도입 추진
향후 축산 농가 마찰 예상 되다
최근 야생 멧돼지
돼지 열병 오다 유사하다 것 분석 되다
발생 흐름 유사하다
경북 북부 지역 확산 우려 되다
상 재화 막 마지막 골든타임 지적
FMD
돼지 열병 박멸 위원회 따르다 야생 멧돼지 돼지 열병 항원
항체 검 추다 지역 이후 경기 연천
충북 충주 포획 되다 야생 멧돼지 돼지 열병 항원
항체 검 추다 면서 내 륙 지역 남하
경기도 강원도 내 지속 적 순환 감염 추정 되다
발생 흐름 이 상당하다 유사하다
북 → 남쪽 방향 점진 적 확산 추세 보이
발생 지역 유추 하다 볼 수 있다
내 륙 지방 ASF 가다 검 추다 되다 가능성 높아지다
이 전문가 들 야생 멧돼지 소탕 등 특단 조치 시
되다 않다 ASF 상 재화 접다 들다
번쨰 기사 이다
바이러스 설악산 이미 오염시키다
소독 등 긴급 방역 조치 실시 하다
가다 등 기존 발생 지점
이번 발견 되다 개체 환경부 설치 울타리 밖 발견 되다 것
설악산 이미 ASF 로 오염 돼다 가능성 높다
시점 이미 감염 되다 개체 추정 되다
설악산 구역 더 남하 지역 홍천
전국 상 재화 단 초 지적 하다
전략 전환 촉구


높다 향후 가정 내 소비 수입 육 비중 더 높아지다 가능성 높다 것 나타나다
이 돼지고기 구입 행태 대한 조사 결과 보다 돼지고기 구입 하다 가구 대다수
조사 돼다
그 비 포장 육 구입 하다 가구
이 g 단위 구입 하다 것 조사 돼다
가구 비다 높다 인 이상 가구 g 비중
구이 용이 특정 부위 쏠리다 더 심하다 것 조사 돼다
저작권 자
경기 북부 지역 생산 성 저하 되다
차량 권역별 이동 제한 모돈 갱신 원활 치 못 영향
어렵다 환경 조성 되다 해당 농가 들 불만 높아지다
발생 이후 포천 도축장 출하
중점 관리 지역 방역 관리 강화 조치 일환 경기도 북부 돼지 타 권역 출하 하다 수 없다 조치 하다 때문
모돈 출하 크다 어려움 겪다
방역 취약점 발생
문턱 가다 돌아서다
초반 대 롤 오가다 부터 대로 떨어지다 후 줄곧 이하 지속 하다
평균 돈 가도 으로 내려앉다
가량 하락 하다
전 년 동월 낮아지다 것 이 처음
상대 적 올해 돼지 값 하락 원인 돼다
대유행 계속 되다 강화 되다 사회 적 거리 두기 유지 되다 등 한 돈 소비 환경 녹다 치 못 하다
감소 하다
이다 지난해 동월 두 대비
감소 하다
저작권 자
올리다 한다는 주장
금요일 경매 활성화 필요하다 지적
한 돈 협회 최근 농협 고령 축산물 공판 장 협회 임원 및 공판 장
관계자 들 참석 가운데 도매시장 활성화 대책 모색 하다
대다 떨어지다 것
중도 매다 비롯 현장 관계자 들 도매시장 활성화 위 하다 고 품질 규격 돈 안정 적 꾸준하다 공급 하다 것 가장 중요하다
문제점 지적 하다
중도 매다 함께 노력 덕분 이전 상황 개선 되어다
밝히다
또 손종 부회장
도매시장 품질 좋다 규격 돈
균일하다 돼지 안정 적 공급 다 면
지역 모범 사례 되다 수 있다
수탁 농가 들 단기간 등락 손해 감수 하다 하다 상황 발생 하다 수 있다
번쨰 기사 이다
돼지고기 수입 량 감소 세 지속 되다
삼겹살 수입 계속 늘
연속 전 년 동월 대비 감소 하다
적다
미국 돼지고기 가격 올해 급등 하다
수입 육 시장 성 낮아지다 것 하다 원인
이처

사업 부진 성장 가능성 낮다 때문 밝히다
하다 계획 것 알려지다
문제 이 도축장 경북 지역 최대 도축 능력 가지다 도축장 점
야기 되다 것 우려 되다
시 장기 적 도인 안정 저해 요소 되다 수 있다 것 지적도 제기 되다
해당 업체 요구
특히 지역 내 도축장 중요성 감안 폐업 시 타 기업 임대 또는 매각 추진 하다 줄 것 요청 하다
농가 들 피해 우려 되다
특히 정부 권역 화 정책 변화 절실 하다 요구 되다
번쨰 기사 이다
방역 시설 전국 양돈 농가 의무 화 되다
전국 확산 되다 우려 증가
전국 양돈 장 방역 강화하다 계획 밝히다
신속하다 방역 시설 강화 필요하다 것 내다보다
법령 개정 신속 추진 키로 하다
설치 농가
지자체 독려 한다는 계획
정부 일방 통행 식 방역 시설 전국 확대 결사 반대 다
마련 하다 것 촉구 하다
시세 역대 돼지 값 중 최고 치르다 기록 하다
오르다
돈 가다 역대 한 돈 시세 역대 최고 가다
연속 역대 동월 최고 가르다 경신 하다 기록 만들다 돼다
하락 세 지속 되다 대까지 떨어지다
에는 대로 재다 입하 강
기조 되찾다
대는 이후 한 달 만이
원인 무엇 공 급량 뚜렷하다 나타나다
줄다
이후 도축 두 수도 크게 늘 못 하다
감소 하다 도 지난해 동월
연휴 예년 같다 비 수기 영향 덜하다 것 파악 되다
저작권 자
양돈 타임스 무단 전 재 및 재 배포 금지 임정은 다른 기사 보기 번쨰 기사 이다
누적 돼지고기 수입량 지난해 물량 처음 추월 하다
수입물량 넘어서다
각각 증가 하다
돼지고기 수입량 톤 지난해 같다 기간 톤
수 입실 적 넘어서다 것 이번 처음
내리다 작년 수입량 뛰어넘다 것
연속 감소 하다 돼지고기 수입량 올해 만에 다시 증가 세로 돌아서다 것 보이다
부위 별로 삼겹살 수입 량 증가 주요 원인 분석 되다
있다
삼겹살 수입량 에도 여톤에 달 하다 전체 수입량
현 시점 지난해 전체 삼겹살 수입 물량 톤에 비 하다 많다
줄다
이후 전 년 대비 증가 세로 돌아서다
지난해 동월 여 톤
감소 하다
감소 반면 스페인 톤
저작권 자
외국인 근로자

In [9]:
### 기사 형태소 분리된 것
df['article_split']=stem_list2

In [96]:
#단어갯수 구하기
embedding_list=[[] for i in range(210)]


In [97]:
for i in df['Week_number'].unique().tolist():
    output = df.loc[df['Week_number'] == i+1, 'article_split']
    embedding_list[i+1].append( output.tolist() )

In [99]:
embedding_list[2]

[[]]

In [56]:
vocab=[]
for i in range(len(embedding_list)-1):
    vocab.append(embedding_list[i].split())

AttributeError: 'list' object has no attribute 'split'

#### 단어 total 갯수 구하기

In [104]:
#단어갯수 구하기
embedding_list2=[]

for week_news in df['article_split']:
    embedding_list2.append(week_news)

In [105]:
vocab=[]
for i in range(len(embedding_list2)):
    vocab.append(embedding_list2[i].split())

In [106]:
vocab_1d=[num for row in vocab for num in row]

In [107]:
vocabulary=set(vocab_1d)

In [110]:
len(vocabulary)

4069

### 주차별로 감성점수 평가

In [112]:
vocab_dic={}
vocab_price={}
vocab_sentiment={}
vocab_sentiment_percent={}

In [113]:
len(embedding_list)

5582

In [114]:
len(vocabulary)

4069

In [ ]:
#(긍정+부정+중립)/전체 단어수
#주차별 키워드 감성점수 평가하기

for i in range(len())

for i in range(len(embedding_list)):
    for word in vocabulary:
        if word in embedding_list[i]:
            if word in vocab_dic:
                    vocab_dic[word] += 1
                    vocab_sentiment[word]+=df['sentiment'].iloc[i]
            else:
                    vocab_dic[word] = 1
                    vocab_sentiment[word]=df['sentiment'].iloc[i]

In [ ]:
#단어별 감성점수 백분위
for word in vocabulary:
    vocab_sentiment_percent[word]=vocab_sentiment[word]/vocab_dic[word]

In [ ]:
dict = sorted(vocab_sentiment_percent.items(), key=lambda x:x[1])

In [120]:
len(embedding_list)

5582